# Install Dependencies

In [ ]:
%pip install comet_ml lightgbm pandas scikit-learn

# Initialize Comet

In [ ]:
import comet_ml

comet_ml.init(project_name="comet-example-lightgbm")

# Import Dependencies

In [ ]:
import os
import pandas as pd
import lightgbm as lgb

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Load Data

In [ ]:
california = fetch_california_housing()
data = pd.DataFrame(california.data)
data.columns = california.feature_names
data["Price"] = california.target
X, y = data.iloc[:, :-1], data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# Train a Model

## Create an Experiment

In [ ]:
experiment = comet_ml.Experiment()

## Define Model Parameters

In [ ]:
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": {"rmse", "l2", "l1", "huber"},
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbosity": -1,
    "early_stopping_rounds": 5,
}

## Fit Model

In [ ]:
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=20,
    valid_sets=[lgb_train, lgb_eval],
    valid_names=("train", "validation"),
)

# End the Experiment

In [ ]:
experiment.end()